<a href="https://colab.research.google.com/github/e96031413/TensorFlow_Learning_2020/blob/master/06_%E7%B7%9A%E6%80%A7%E8%BF%B4%E6%AD%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#交叉驗證

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

np.random.seed(7)  # 指定亂數種子
# 載入波士頓房屋資料集
df = pd.read_csv("./boston_housing.csv")
dataset = df.values
np.random.shuffle(dataset)  # 使用亂數打亂資料
# 分割成特徵資料和標籤資料
X = dataset[:, 0:13]
Y = dataset[:, 13]
# 特徵標準化
X -= X.mean(axis=0)
X /= X.std(axis=0)
# 分割訓練和測試資料集
X_train, Y_train = X[:404], Y[:404]     # 訓練資料前404筆
X_test, Y_test = X[404:], Y[404:]       # 測試資料後102筆
# 定義模型
def build_deep_model():
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation="relu"))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1))
    # 編譯模型
    model.compile(loss="mse", optimizer="adam", 
                  metrics=["mae"])
    return model

k = 4
nb_val_samples = len(X_train) // k
nb_epochs = 80
mse_scores = []
mae_scores = []
for i in range(k):
    print("Processing Fold #" + str(i))
    # 取出驗證資料集
    X_val = X_train[i*nb_val_samples: (i+1)*nb_val_samples]
    Y_val = Y_train[i*nb_val_samples: (i+1)*nb_val_samples]
    # 結合出訓練資料集
    X_train_p = np.concatenate(
            [X_train[:i*nb_val_samples],
            X_train[(i+1)*nb_val_samples:]], axis=0)
    Y_train_p = np.concatenate(
            [Y_train[:i*nb_val_samples],
            Y_train[(i+1)*nb_val_samples:]], axis=0)
    model = build_deep_model()
    # 訓練模型
    model.fit(X_train_p, Y_train_p, epochs=nb_epochs, 
              batch_size=16, verbose=0)
    # 評估模型
    mse, mae = model.evaluate(X_val, Y_val)
    mse_scores.append(mse)
    mae_scores.append(mae)
    
print("MSE_val: ", np.mean(mse_scores))
print("MAE_val: ", np.mean(mae_scores))
# 使用測試資料評估模型
mse, mae = model.evaluate(X_test, Y_test)    
print("MSE_test: ", mse)
print("MAE_test: ", mae)


Processing Fold #0
101/101 [==============================] - 0s 704us/step
Processing Fold #1
101/101 [==============================] - 0s 903us/step
Processing Fold #2
101/101 [==============================] - 0s 1ms/step
Processing Fold #3
101/101 [==============================] - 0s 1ms/step
MSE_val:  18.426768534254318
MAE_val:  2.846781608491841
102/102 [==============================] - 0s 60us/step
MSE_test:  8.395139236076206
MAE_test:  2.166057037372215


In [23]:
#無交叉驗證

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

np.random.seed(7)  # 指定亂數種子
# 載入波士頓房屋資料集
df = pd.read_csv("./boston_housing.csv")
dataset = df.values
np.random.shuffle(dataset)  # 使用亂數打亂資料
# 分割成特徵資料和標籤資料
X = dataset[:, 0:13]
Y = dataset[:, 13]
# 特徵標準化
X -= X.mean(axis=0)
X /= X.std(axis=0)
# 分割訓練和測試資料集
X_train, Y_train = X[:404], Y[:404]     # 訓練資料前404筆
X_test, Y_test = X[404:], Y[404:]       # 測試資料後102筆
# 定義模型
model = Sequential()
model.add(Dense(32, input_shape=(X_train.shape[1],), activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(1))
# 編譯模型
model.compile(loss="mse", optimizer="adam", 
              metrics=["mae"])
# 訓練模型
model.fit(X_train, Y_train, epochs=80, batch_size=16, verbose=0)
# 使用測試資料評估模型
mse, mae = model.evaluate(X_test, Y_test)    
print("MSE_test: ", mse)
print("MAE_test: ", mae)

102/102 [==============================] - 0s 521us/step
MSE_test:  8.12880017710667
MAE_test:  2.142300393067154


In [0]:
# 儲存神經網路模型結構與權重

## 方法1
json_str = model.to_json()
with open("Model.config", "w") as text_file:
  text_file.write(json_str)
model.save_weights("Model.weight")

## 方法2
model.save("Model.h5")  #將模型結構與權重保存成單一檔案(HDF5格式*.h5)

In [0]:
# 載入神經網路模型結構與權重

##  方法1(分開載入)
from keras.models import model_from_json

model = Sequential()
with open("Model.config", "r") as text_file:
  json_str = text_file.read()
model = model_from_json(json_str)

model.load_weights("Model.weight", by_name=False)           #載入權重後還要compile()才能評估模型和計算預測值
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

##  方法2(一次載入)
from keras.models import load_model

model = Sequential()
model = load_model("Model.h5")
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])